In [4]:
import yfinance as yf
import pandas as pd
import plotly.express as px

# Definindo o período de análise
start_date = "2023-01-01"
end_date = "2024-01-01"

# Solicitando o ticker de referência
ticker = str(input('\nDigite o ticker Referência: ') + ".SA").upper()

# Baixando dados do ativo e do IBOV
df0 = yf.download(ticker, start=start_date, end=end_date)
df1 = yf.download("^BVSP", start=start_date, end=end_date)

# Calculando o retorno diário
df0['Return'] = df0['Close'].pct_change()
df1['Return'] = df1['Close'].pct_change()

# Calculando a força relativa
df0['RS'] = df0['Return'].rolling(window=10).mean() - df1['Return'].rolling(window=10).mean()

# Calculando o momentum da força relativa
df0['Momentum'] = df0['RS'].diff()

# Criando DataFrame para plotagem e removendo NaN
data = pd.DataFrame({
    'Date': df0.index,
    'RS': df0['RS'],
    'Momentum': df0['Momentum']
}).dropna()

# Convertendo a coluna de datas para string
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')

# Criando o gráfico interativo com plotly
fig = px.scatter(
    data,
    x='RS',
    y='Momentum',
    animation_frame='Date',
    range_x=[data['RS'].min(), data['RS'].max()],
    range_y=[data['Momentum'].min(), data['Momentum'].max()],
    title=f'Relative Rotation Graph (RRG) for {ticker}',
    labels={'RS': 'Relative Strength', 'Momentum': 'Momentum'},
    template='plotly_dark',
    size_max=50,  # Tamanho máximo dos pontos
    opacity=0.8,
    color_discrete_sequence=['white'],  # Cor dos pontos
)

# Adicionando linhas para indicar os quadrantes
fig.add_shape(
    type='line',
    x0=data['RS'].min(),
    x1=data['RS'].max(),
    y0=0,
    y1=0,
    line=dict(color='yellow', dash='dash'),
)
fig.add_shape(
    type='line',
    x0=0,
    x1=0,
    y0=data['Momentum'].min(),
    y1=data['Momentum'].max(),
    line=dict(color='yellow', dash='dash'),
)

# Ajustando layout para melhorar a animação e aumentar o tamanho
fig.update_layout(
    transition={'duration': 500},
    xaxis_title='Relative Strength',
    yaxis_title='Momentum',
    width=1000,  # Largura do gráfico
    height=800,  # Altura do gráfico
)

fig.show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
